In [ ]:
# 从每2张切片上随机抽取30个roi,按顺序排列。不排列也可以，计算均值部分用get_index1函数即可。
import random

def get_index1(lst=None, item=''):
    """只能抽取纯字符串形式"""
    return [index for (index,value) in enumerate(lst) if (item in value)]
fl20 = []# fl20是抽取后的文件名
for i1 in range(2,27,2):
    day = "day" + str(i1)
    # fl是所有文件名
    indexc = get_index1(fl, day+" c")
    indext = get_index1(fl, day+" T")
    for i2 in random.sample(indexc, 30):
        fl20.append(fl[i2])   
    for i2 in random.sample(indext, 30):
        fl20.append(fl[i2])  
print(len(fl20))

In [ ]:
def Calculate_pixnum(Cluster_1 ,roi_num=31):
    """输入roi数量及超像素点序号，返回具体的像素点个数
    """
    pt = 0
    pixnum = []

    for t in range(0,roi_num*1024+1):
    #     if t < 3:
    #         continue
        if (t%1024)==0:
            ind = t//1024 - 1
            inx = ind + 1
            if inx < roi_num:
                sup_pix = loadmat(fl[inx]+'/no_emp_km_labels.mat')['ne_km_labels']
            if t!=0:
#                 print('\n '+ str(t//1024-1)+':'+fl[ind])
#                 print(pt)
                pixnum.append(pt)
            pt = 0
        if t in Cluster_1:
            pix_num = len(zeros(sup_pix.shape[:2])[sup_pix==(t%1024)])
            pt += pix_num
    return pixnum

In [ ]:
# 根据我的实测，看13个点的趋势的情况下，超像素的变化与具体像素点的数量差别并不明显。
# 计算780个roi，超像素计算时间为3Min，具体像素点计算时间为3H
# 便于排序，我把样本顺序排列了。如果样本没有顺序排列，可以用第一个cell里的get_index1获取索引列表
Cmeans = []
Tmeans = []
cluster_labels = km.labels_.copy()# 降维聚类后的标记点

for i in range(0, cluster_labels.max()+1):
    obj_clt = i
    Cluster_1 = []
    Cmeans.append([])
    Tmeans.append([])
    for i in range(len(cluster_labels)):
        if cluster_labels[i] == obj_clt:
            Cluster_1.append(i)
# 算具体像素点数，样本是按顺序排列的
    pixnum = Calculate_pixnum(Cluster_1,roi_num=len(fl))
    for ci in range(0,13,1):
        Cmeans[obj_clt].append(np.mean(pixnum[ci*60:60*ci+30]))# 如果未按序排列，这里用get_index1替换
        Tmeans[obj_clt].append(np.mean(pixnum[60*ci+30:60*ci+60]))# 同上一行
# 算超像素个数
#     spixnum = np.zeros((53))
#     for point in Cluster_1:
#         spixnum[point//(1024*30)] = spixnum[point//(1024*30)]+1
#     for ci in range(0,26,2):
#         Cmeans[obj_clt].append(spixnum[ci])
#         Tmeans[obj_clt].append(spixnum[ci+1])

In [ ]:
# 画图
for clusternum in range(0,6):# 这个类别是分类的数量
    roipix = fin_img.shape[0]*fin_img.shape[1]
    ratiox = np.zeros((27))# 样本天数
    for dayi in range(2,26,2):
        ratiox[dayi] = Cmeans[clusternum][dayi//2-1]
        
    timex = np.arange(0,26,2)
    my_x_ticks = np.arange(0,26,2)
    plt.xticks(my_x_ticks)
    plt.plot(timex,np.array(Cmeans[clusternum][::-1]), label='Control')
    
    ratiox = np.zeros((26))
    for dayi in range(2,26,2):
        ratiox[dayi] = Tmeans[clusternum][dayi//2-1]
#     plt.plot(timex,array(Tmeans[clusternum][::-1])/roipix, label='T')
    plt.plot(timex,np.array(Tmeans[clusternum][::-1]), label='T')
    
    
    ticklabel = ['0','1','2','3','4','5']
    plt.title(ticklabel[clusternum])
    plt.legend()
    plt.savefig("G:/6cluster/"+ticklabel[clusternum]+".jpg")# 这里的文件名注意更改下
    plt.close()